# Automation for Webserver, MySQL and MongoDB

## Create 2 EC2 instances 

In [ ]:
import boto3
import os
import time
from fabric import Connection
from botocore.exceptions import ClientError
ec2 = boto3.client('ec2')

### 1. Functions

In [ ]:
instance_ami = 'ami-061eb2b23f9f8839c'
num_instance = 1
tier = 't2.micro'
key_pair = 'prodkey21'
security_group_name = 'sum'
security_desc = 'Security group for Mongo'

In [ ]:
# AWS EC2 list instance images
def list_images():
    res = ec2.describe_images(Owners=['self'])
    for img in res['Images']:
        print("Name: ",img['Name'])
        print("Image: ", img['ImageId'])
        print("Description: ", img['Description'])
        print("----")

In [ ]:
# AWS EC2 save instance image
def save_instance(ins, name, desc='My new instance'):
    res = ec2.create_image(InstanceId=ins, Name=name, Description=desc)
    print("Created image: ",res['ImageId'])
    print("Waiting for it to be available...")

    # wait for it to be available
    available = 0
    while (not available):
        status = ec2.describe_images(ImageIds=[res['ImageId']])
        img = status['Images'][0]
        available = (img['State'] == 'available')
        time.sleep(1)

In [ ]:
# AWS EC2 create key-pair
def create_key_pair(name):
    response = ec2.create_key_pair(
        KeyName=name
    )
    key = response['KeyMaterial']
    f= open(name+".pem","w")
    f.write(key)
    f.close
    os.system('chmod 400 %s.pem' %name)
    print('Key Pair %s Created' %name)
    return response

In [ ]:
# AWS EC2 delete key-pair
def delete_key_pair(name):
    ec2.delete_key_pair(KeyName=name)
    os.remove(name+'.pem')
    print('Key Pair %s Deleted' %name)

In [ ]:
# AWS EC2 create security group
def create_security_group(name, description, ip_permissions):
    response = ec2.describe_vpcs()
    vpc_id = response.get('Vpcs', [{}])[0].get('VpcId', '')

    response = ec2.create_security_group(GroupName=name,
                                         Description=description,
                                         VpcId=vpc_id)
    security_group_id = response['GroupId']
    print('Security Group Created %s in vpc %s.' % (security_group_id, vpc_id))

    data = ec2.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions=ip_permissions)
    print('Ingress Successfully Set %s' %data)
    return security_group_id

In [ ]:
# AWS EC2 remove Security group
def remove_security_group_permissions(security_group_id, ip_permissions):
    data = ec2.revoke_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions=ip_permissions)
    print('Ingress Successfully removed %s' %data)
    return security_group_id

In [ ]:
# AWS EC2 set Security group permissions
def set_security_group_permissions(security_group_id, ip_permissions):
    data = ec2.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions=ip_permissions)
    print('Ingress Successfully Set %s' %data)
    return security_group_id

In [ ]:
# AWS EC2 delete security group
def delete_security_group(security_group_id):
    response = ec2.delete_security_group(GroupId=security_group_id)
    print('Security Group %s Deleted' %security_group_id)

In [ ]:
# AWS EC2 create instances
def create_instances(image, maxCount, instanceType, key, groupID):
    instances = ec2.run_instances(
        ImageId=image,
        MinCount=1,
        MaxCount=maxCount,
        InstanceType=instanceType,
        KeyName=key,
        SecurityGroupIds=[groupID]
    )
    instance_list = []
    for i in instances['Instances']:
        instance_list.append(i['InstanceId'])
    
    print('Instances Created %s' %instance_list)
    return instance_list

In [ ]:
# AWS EC2 list instances
def list_ec2_instances():
    instances = {}
    res = ec2.describe_instances()
    for r in res['Reservations']:
        for ins in r['Instances']:
            if ins['State']['Name'] == 'running' or ins['State']['Name'] == 'pending':
                instances[ins['InstanceId']] = ins['PublicIpAddress']
    print('List of active Instances %s' %instances)
    return instances

In [ ]:
# AWS EC2 terminate instances
def terminate_instances(instances):
    response = ec2.terminate_instances(
        InstanceIds=instances
    )
    print('Instances Deleted %s' %instances)
    return response

### 2. Generate Key pair 

In [ ]:
# generate the new key pair
# create_key_pair(key_pair)

## Set up Ubuntu Node for EC2 instance

### 1. Set up Security group 

In [ ]:
# set up permissions for the new security group for the instance/node
node_ip_permissions = [{'IpProtocol': 'tcp',
                   'FromPort': 22,
                   'ToPort': 22,
                   'IpRanges': [{
                       'CidrIp': '0.0.0.0/0',
                       'Description': 'SSH'}]},
                  {'IpProtocol': '-1',
                   'IpRanges': [{
                       'CidrIp': '0.0.0.0/0',
                       'Description': 'All'}]},
                  {'IpProtocol': 'tcp',
                   'FromPort': 27017,
                   'ToPort': 27017,
                   'IpRanges': [{
                       'CidrIp':'0.0.0.0/0',
                       'Description': 'MongoDB'}]}]

In [ ]:
# create the security group using the previously made permissions
node_groupID = create_security_group(security_group_name, security_desc, node_ip_permissions)
node_groupID

In [ ]:
# create a new single instance
# we are using the ubuntu AMI for this and using t2.medium
instance_node_list = create_instances(instance_ami, num_instance, tier, key_pair, node_groupID)
instance_node = instance_node_list[0]
instance_node

In [ ]:
# get the public IP of the instance
print('Creating EC2 Instance...')
time.sleep(5)
instance_dic = list_ec2_instances()
node_ip = instance_dic[instance_node]
node_ip

## A. Setting up Metadata

In [ ]:
# Installing Mongo into EC2 instance
print('EC2 instance rebooting, please wait...')
print('Approx. waiting time: 2-5 mins')
time.sleep(50)
mongo_ip = '22.22.222' # for example
name_of_db = 'nezukodb'
mongo_url = f'mongodb://{mongo_ip}' # insert mongo url here

# write to your .env file
environment_file = open(".env", 'w')

environment_file.write(f'database_name = {name_of_db}\n')
environment_file.write(f'mongo_url = {mongo_url}\n')

environment_file.close()


c.sudo('apt-get install gnupg')
c.run('wget -qO - https://www.mongodb.org/static/pgp/server-4.2.asc | sudo apt-key add -')
c.run('echo "deb [ arch=amd64 ] https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.2 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-4.2.list')
c.sudo('apt-get update')
c.sudo('apt-get install -y mongodb-org')
c.sudo('service mongod start')

# Transfer Metadata from S3 to the instance
c.run('wget https://dbds-kindle-reviews.s3-ap-southeast-1.amazonaws.com/metadata.bson')
c.run(f'mongorestore --db {name_of_db} metadata.bson')
c.sudo("sed -i 's/127.0.0.1/0.0.0.0/g' /etc/mongod.conf")
c.sudo('service mongod restart')

## B. Setting up MySQL Kindle Reviews

In [ ]:
print('Setting up EC2 instance, please wait...')
print('Approx. waiting time: 2-5 mins')
print('Note: You might wanna go to the toilet first or make a hot coffee while waiting.')
time.sleep(50)
c.sudo('apt-get update')
c.sudo('apt-get -y dist-upgrade')

In [ ]:
# rebooting

try:
    ec2.reboot_instances(InstanceIds=[instance_node], DryRun=True)
except ClientError as e:
    if 'DryRunOperation' not in str(e):
        print("You don't have permission to reboot instances.")
        raise

try:
    response = ec2.reboot_instances(InstanceIds=[instance_node], DryRun=False)
    print('Success', response)
except ClientError as e:
    print('Error', e)

In [ ]:
# Installing MySQL

print('EC2 instance rebooting, please wait...')
print('Approx. waiting time: 2-5 mins')
print('You know what to do.')
time.sleep(50)
c.sudo('apt-get -o StrictHostKeyChecking=no update')
c.sudo('apt-get dist-upgrade')
c.sudo('apt-get -y install mysql-server')

In [ ]:
# Transfer kindle_reviews.csv from S3 to the instance

c.run('mkdir data') 
c.run('cd data && wget -c https://dbds-kindle-reviews.s3-ap-southeast-1.amazonaws.com/kindlereviews.sql')

In [ ]:
# Setting up kindle_reviews database

c.sudo('mysql -e "create database dbds"')
c.sudo('mysql -e "' + 'GRANT ALL PRIVILEGES ON *.* TO' + "'ubuntu'" + 'IDENTIFIED BY' + "'password';" + '"')
c.run('cd data && mysql -u ubuntu -ppassword -D dbds -e "source kindlereviews.sql"')
c.sudo("sed -i 's/bind-address/#bind-address/g' /etc/mysql/mysql.conf.d/mysqld.cnf")
c.sudo('service mysql restart')

## C. Setting up Web Server 

In [ ]:
# Importing Project Repo from Github

print('Setting up EC2 instance, please wait...')
print('Approx. waiting time: 2-5 mins')
time.sleep(65)
c.run('mkdir dbds_project')
c.run('cd dbds_project && git clone https://github.com/hello2508/Database.git')

In [ ]:
# Downloading the necessary packages

print('Updating...')
c.sudo('apt-get update')
c.sudo('apt-get install python3')

In [ ]:
# Install prerequisites before installing pip3

c.sudo('apt-get -y install build-essential libssl-dev libffi-dev python-dev')
c.sudo('apt-get update')
c.sudo('apt -y install python3-pip')

In [ ]:
# Create a virtual environment through Python3-venv

print('Creating virtual environment(webenv) for web server...')
c.sudo('apt install -y python3-venv')
c.run('cd dbds_project/Database/flaskproject && python3 -m venv webvenv')
time.sleep(5)
c.run('cd dbds_project/Database/flaskproject && source ./webvenv/bin/activate')

In [ ]:
# Get dependencies

print('Installing relevant packages...')
c.sudo('pip3 install --upgrade setuptools')
c.sudo('apt-get -y install python3-dev libmysqlclient-dev')
c.sudo('apt -y install libmpc-dev')
c.run('pip3 install --user gmpy2==2.1.0a2')
c.run('cd dbds_project/Database && pip3 install -r requirements.txt')
c.sudo('pip3 install pymongo')
c.sudo('apt-get install python3-mysql.connector')

In [ ]:
# Running Flask (ensure this is running in the correct directory)

time.sleep(3)
print('Setting up web server...')
print('Once completed, pls go to localhost:8888 to see the Web server.')
c.run('cd dbds_project/Database/flaskproject && export FLASK_APP=app.py')
c.run('cd dbds_project/Database/flaskproject && export FLASK_ENV=development')
c.run('cd dbds_project/Database/flaskproject && python3 app.py')